In [1]:
import numpy as np
import pandas as pd
import scipy
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader,random_split,sampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from PIL import Image
from tqdm.auto import tqdm
import timm
import sklearn
from sklearn.metrics import balanced_accuracy_score
import torchvision
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [4]:
train_csv = pd.read_csv('/kaggle/input/UBC-OCEAN/train.csv')
test_csv = pd.read_csv('/kaggle/input/UBC-OCEAN/test.csv')

In [5]:
print(train_csv)

     image_id label  image_width  image_height  is_tma
0           4  HGSC        23785         20008   False
1          66  LGSC        48871         48195   False
2          91  HGSC         3388          3388    True
3         281  LGSC        42309         15545   False
4         286    EC        37204         30020   False
..        ...   ...          ...           ...     ...
533     65022  LGSC        53355         46675   False
534     65094    MC        55042         45080   False
535     65300  HGSC        75860         27503   False
536     65371  HGSC        42551         41800   False
537     65533  HGSC        45190         33980   False

[538 rows x 5 columns]


In [6]:
train_csv_tma = train_csv.loc[train_csv['is_tma']==True]
print(train_csv_tma)

     image_id label  image_width  image_height  is_tma
2          91  HGSC         3388          3388    True
37       4134    MC         2964          2964    True
76       8280  HGSC         2964          2964    True
83       9200    MC         3388          3388    True
112     13568  LGSC         2964          2964    True
149     17637  HGSC         2964          2964    True
176     21020    MC         3388          3388    True
236     29084  LGSC         3388          3388    True
263     31594    EC         3388          3388    True
288     35565    MC         2964          2964    True
299     36302    CC         3388          3388    True
302     36583  LGSC         3388          3388    True
305     36783    MC         2964          2964    True
309     37385  LGSC         3388          3388    True
350     40864  LGSC         2964          2964    True
354     41368    EC         3388          3388    True
355     41586    CC         2964          2964    True
361     42

In [7]:
train_csv_ntma = train_csv.loc[train_csv['is_tma']==False]
print(train_csv_ntma)

     image_id label  image_width  image_height  is_tma
0           4  HGSC        23785         20008   False
1          66  LGSC        48871         48195   False
3         281  LGSC        42309         15545   False
4         286    EC        37204         30020   False
5         431  HGSC        39991         40943   False
..        ...   ...          ...           ...     ...
533     65022  LGSC        53355         46675   False
534     65094    MC        55042         45080   False
535     65300  HGSC        75860         27503   False
536     65371  HGSC        42551         41800   False
537     65533  HGSC        45190         33980   False

[513 rows x 5 columns]


In [8]:
##We can observe tma images have upper bound of 40,000*40,000 and non_tma have range a lot greater than that
## Plan, height>=10k or weight>=10k,it's ntma else it's tma
##Get avg width and height of each image
print(train_csv_ntma['image_width'].mean())
print(train_csv_ntma['image_height'].mean())
#for tma resize to 4k*4k is safe and for ntma resize to 50k*50k is safe

51084.586744639375
31022.249512670565


In [9]:
train_csv['label'].value_counts()

label
HGSC    222
EC      124
CC       99
LGSC     47
MC       46
Name: count, dtype: int64

In [10]:
train_csv_tma['label'].value_counts()

label
HGSC    5
MC      5
LGSC    5
EC      5
CC      5
Name: count, dtype: int64

In [11]:
train_csv_ntma['label'].value_counts()

label
HGSC    217
EC      119
CC       94
LGSC     42
MC       41
Name: count, dtype: int64

In [12]:
transform_tma = A.Compose(
[
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ColorJitter(),
    A.Resize(4000,4000),
    A.Normalize(mean=(0.485,0.456,0.406),std=(0.229,0.224,0.225)),
    ToTensorV2()
]
)
transform_ntma = A.Compose(
[
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ColorJitter(),
    A.Resize(3000,3000),
    A.Normalize(mean=(0.485,0.456,0.406),std=(0.229,0.224,0.225)),
    A.PadIfNeeded(min_height=4000,min_width=4000,border_mode=cv2.BORDER_CONSTANT,value=(0,0,0)),
    ToTensorV2()
]
)

In [13]:
train_csv['label'] = train_csv['label'].map({"CC":0,"EC":1,"HGSC":2,"LGSC":3,"MC":4})

In [14]:
class dataset(Dataset):
    def __init__(self,transform=(None,None)):
        self.dir1 = '/kaggle/input/UBC-OCEAN/train_images/'
        self.dir2 = '/kaggle/input/UBC-OCEAN/train_thumbnails/'
        self.transform_tma = transform[0]
        self.transform_ntma = transform[1]
    def __len__(self):
        return len(os.listdir(self.dir1))
    def __getitem__(self,idx):
        img_id = train_csv['image_id'][idx]
        if(train_csv['is_tma'][idx]):
            img = cv2.imread(self.dir1+str(img_id)+'.png')
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img = self.transform_tma(image=img)
            img_type = 1
        else:
            img = cv2.imread(self.dir2+str(img_id)+'_thumbnail.png')
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img = self.transform_ntma(image=img)
            img_type = 0
        return{
            'image':img['image'],
            'img_type':img_type,
            'label':train_csv['label'][idx]
        }

In [15]:
total_data = dataset(transform=(transform_tma,transform_ntma))

In [16]:
train_ratio = 0.9
val_ratio = 0.1
total_size = len(total_data)
train_size = int(total_size*train_ratio)
val_size = total_size-train_size
train_data,val_data = random_split(total_data,[train_size,val_size])

In [17]:
base_model = timm.create_model('resnet101',pretrained=True)
last_layer = nn.Sequential(
    nn.Linear(in_features=base_model.num_features,out_features = 1000),
    nn.BatchNorm1d(1000),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(1000,6)
)
base_model.fc = last_layer

In [18]:
tma_layer = nn.Sequential(
    nn.Conv2d(3,32,(7,7),(5,5)),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Conv2d(32,128,(6,6),(4,4),50),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128,3,(1,1)),
    nn.BatchNorm2d(3),
    nn.ReLU()
)
ntma_layer = nn.Sequential(
    torchvision.transforms.CenterCrop(3000),
    nn.Conv2d(3,32,(7,7),(5,5)),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Conv2d(32,128,(5,5),(3,3),38),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128,3,(1,1)),
    nn.BatchNorm2d(3),
    nn.ReLU()
)

In [19]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.tma = tma_layer
        self.ntma = ntma_layer
        self.base = base_model
    
    def forward(self,x,img_type):
        tma_indices = img_type==1
        ntma_indices = img_type==0
        tma_inputs = x[tma_indices]
        ntma_inputs = x[ntma_indices]
        tma_outputs = self.tma(tma_inputs) if tma_inputs.size(0)>0 else torch.empty(0).to(device)
        ntma_outputs = self.ntma(ntma_inputs) if ntma_inputs.size(0)>0 else torch.empty(0).to(device)
        emp = torch.cat((tma_outputs,ntma_outputs))
        out = self.base(emp)
        return out

In [20]:
my_model = Model()
my_model.to(device)
#Sample pass of created model 
my_model.eval()
sample_pass_img = total_data[0]['image']
sample_pass_type = torch.tensor([total_data[0]['img_type']])
sample_pass_img = sample_pass_img[None,:,:,:]
sample_pass_img = sample_pass_img.to(device)
print(my_model(sample_pass_img,sample_pass_type))
torch.cuda.empty_cache()

tensor([[-0.0385,  0.0193,  0.0433, -0.0254, -0.0195,  0.0047]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [21]:
classes = np.array(train_csv["label"])
class_weights = sklearn.utils.class_weight.compute_class_weight('balanced',classes=np.unique(classes),y=classes)

In [22]:
weighted_sampler = sampler.WeightedRandomSampler(weights=class_weights,num_samples=train_size,replacement=True)
train_loader = DataLoader(train_data,batch_size=4)
val_loader = DataLoader(val_data,batch_size=4,shuffle=False)

In [23]:
def train_func(train_loader,model,optimizer,criterion):
    model.train()
    epoch_loss=0
    count=0
    preds = torch.empty((0,6)).cpu()
    labels = torch.empty((0)).cpu()
    for i,data in enumerate(tqdm(train_loader,total=len(train_loader))):
        images = data['image'].to(device)
        label = torch.tensor(data['label']).to(device)
        img_type = torch.tensor(data['img_type']).to(device)
        output = model(images,img_type)
        loss = criterion(output,label)
        loss.backward()
        optimizer.step()
        preds = torch.cat((preds,output.detach().cpu()))
        labels = torch.cat((labels,label.detach().cpu()))
        epoch_loss += loss.item()*images.shape[0]
        count+=images.shape[0]
    with torch.no_grad():
        preds = nn.Softmax(dim=1)(preds)
        preds = torch.argmax(preds,dim=1)
        score = balanced_accuracy_score(labels,preds)
    return epoch_loss/count,score

def val_func(val_loader,model,criterion):
    model.eval()
    loss=0
    count=0
    preds = torch.empty((0,6)).cpu()
    labels = torch.empty((0)).cpu()
    for i,data in enumerate(tqdm(val_loader,total=len(val_loader))):
        images = data['image'].to(device)
        label = torch.tensor(data['label']).to(device)
        img_type = torch.tensor(data['img_type']).to(device)
        with torch.no_grad():
            output = model(images,img_type)
            step_loss = criterion(output,label)
            preds = torch.cat((preds,output.detach().cpu()))
            labels = torch.cat((labels,label.detach().cpu()))
            loss += step_loss.item()*images.shape[0]
            count+=images.shape[0]
    with torch.no_grad():
        preds = nn.Softmax(dim=1)(preds)
        preds = torch.argmax(preds,dim=1)
        score = balanced_accuracy_score(labels,preds)
    return loss/count,score

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(my_model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=2)

In [25]:
#Training loop
best_vscore = 0
epochs=5
for epoch in range(epochs):
    train_loss,tscore = train_func(train_loader,my_model,optimizer,criterion)
    val_loss,vscore = val_func(val_loader,my_model,criterion)
    print(f"Epoch{epoch+1} train_loss:{train_loss:.4f}",f"train_score:{tscore:.4f}")
    print(f"Epoch{epoch+1} val_loss:{val_loss:.4f}",f"val_score:{vscore:.4f}")
    scheduler.step()
    if vscore>best_vscore:
        print("Validation Accuracy Improved")
        best_vscore = vscore
        best_model = my_model

  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

Epoch1 train_loss:5.2702 train_score:0.2158
Epoch1 val_loss:19.4009 val_score:0.1809
Validation Accuracy Improved


  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

...

  0%|          | 0/14 [00:00<?, ?it/s]

Epoch4 train_loss:4.0534 train_score:0.1906
Epoch4 val_loss:8.6910 val_score:0.2180
Validation Accuracy Improved


  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

Epoch5 train_loss:3.6117 train_score:0.2343
Epoch5 val_loss:12.9025 val_score:0.1320


In [26]:
torch.save(my_model.state_dict(),"model.pt")

In [27]:
def test_func(test_loader,model,device):
    model.to(device).eval()
    predictions = np.zeros((0,6))
    for i,data in enumerate(tqdm(test_loader,total = len(test_loader))):
        images = data['image'].to(device)
        img_type = torch.tensor(data['img_type']).to(device)
        with torch.no_grad():
            output = model(images,img_type)
            predictions = np.concatenate((predictions,output.softmax(1).to('cpu').numpy()),axis=0)
    predictions = np.argmax(predictions,axis=1)
    return predictions    

In [28]:
class testDataset(Dataset):
    def __init__(self,transform=(None,None)):
        self.dir1 = "/kaggle/input/UBC-OCEAN/test_images/"
        self.dir2 = "/kaggle/input/UBC-OCEAN/test_thumbnails/"
        self.transform_tma = transform[0]
        self.transform_ntma = transform[1]
    def __len__(self):
        return len(os.listdir(self.dir1))
    def __getitem__(self,idx):
        img_id = test_csv['image_id'][idx]
        h = test_csv['image_height'][idx]
        w = test_csv['image_width'][idx]
        if((h<10000)and(w<10000)):
            img = cv2.imread(self.dir1+str(img_id)+'.png')
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img = self.transform_tma(image=img)
            img_type = 1
        else:
            img = cv2.imread(self.dir2+str(img_id)+'_thumbnail.png')
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img = self.transform_ntma(image=img)
            img_type = 0
        return{
            'image':img['image'],
            'img_type':img_type,
        }

In [29]:
test_dataset = testDataset(transform=(transform_tma,transform_ntma))
test_loader = DataLoader(test_dataset,batch_size=4,shuffle=False)
pred = test_func(test_loader,best_model,device)

  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
print(pred)

[0]


In [31]:
sub = pd.read_csv("/kaggle/input/UBC-OCEAN/test.csv")
sub = sub.drop(["image_width","image_height"],axis=1)
sub["label"] = pred
sub['label'] = sub['label'].map({0:"CC",1:"EC",2:"HGSC",3:"LGSC",4:"MC",5:"Other"})
sub

,image_id,label
0,41,CC


In [32]:
sub.to_csv('submission.csv',index=False)